In [1]:
import numpy as np
import pandas as pd
import codecs
import matplotlib
import matplotlib.pyplot as plt
import random
import random
import codecs
# this input_path is just example
#inputpath = "C:/Users/Administrator/Desktop/Smote/describledata/kddcup.data_10_percent_corrected"
Nor_list = ['normal']
Dos_list = ['back', 'land', 'neptune' , 'pod', 'smurf', 'teardrop', 'mailbomb', 'processtable', 'udpstorm', 'apache2', 'worm']
R2L_list =['guess_passwd', 'ftp_write', 'imap', 'phf', 'multihop', 'warezmaster', 'warezclient', 'xlock','xsnoop', 'snmpguess', 'snmpgetattack', 'httptunnel','spy','named','snmpguess']
Probe_list = ['satan','ipsweep', 'nmap', 'portsweep', 'mscan', 'saint']
U2R_list =['buffer_overflow', 'loadmodule', 'rootkit', 'perl', 'sqlattack', 'xterm', 'ps','httptunnel']
continue_col_index = [0,4,5,7,8,9,10,12,15,16,17,18,19,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]
decreate_col_index =[1,2,3,6,11,13,14,20,21]

In [2]:
def gen_lable1(seg):
    if seg in Nor_list:
        return  0
    elif seg in Dos_list:
        return  1
    elif seg in R2L_list:
        return  2
    elif seg in Probe_list:
        return 3
    elif seg in U2R_list:      
        return  4
    
    
def gen_label2(seg):
    if seg in Nor_list:
        return  0
    else:
        return 1
def replace(columnlist,data):
    #离散值代替
    #listt = []
    for i in columnlist:
        listt = []
        for line,seg in enumerate(data[:,i]):
            if seg in listt:
                data[line][i] = listt.index(seg)
            else:
                listt.append(seg)
                data[line][i] = listt.index(seg)

def processdata(file_path):
    tmp = None
    with codecs.open(file_path,'r') as f:
        content = f.readlines()
        datas =[]
        for line in content:
            line = line.strip()
            line = line[:-1].split(',')
            datas.append(line)
        datas = np.array(datas)
        replace(decreate_col_index,datas)
        new_datas =[]
        for index,col in enumerate(datas):
            new_datas.append([float(k) for k in col[:-1]])
            if random.random()<0.00005:
                print(col[-1])
            new_datas[index].append(gen_lable1(col[-1]))
            new_datas[index].append(gen_label2(col[-1]))
        tmp = new_datas
    datas = np.array(tmp).astype('float32')
    
    for j in continue_col_index:
        meanVal=np.mean(datas[:,j])
        stdVal=np.std(datas[:,j])
        datas[:,j]=(datas[:,j]-meanVal)/stdVal
    return datas


In [3]:
data = processdata('./describledata/kddcup.data_10_percent_corrected')

normal


normal


normal
smurf


neptune


neptune


...

neptune


neptune


smurf


smurf
smurf
normal


C:\Users\Administrator\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in true_divide


In [4]:
#
label_dict ={}
for i in data[:,-2]:
    if i in label_dict.keys():
        continue
    else:
        label_dict[i] = len(label_dict)
test_data = processdata('./describledata/corrected')

smurf
smurf


normal


normal


neptune
neptune


smurf


smurf


smurf


neptune
neptune


C:\Users\Administrator\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in true_divide


In [5]:
def fill_nan(data):
    #m,n = data.shape
    choice_list =[]
    matrix = np.isnan(data)
    for m in range(len(data)):
        for n in range(len(data[0])):
            if matrix[m,n] ==True:
                data[m][n] = 0

In [6]:
fill_nan(data)
fill_nan(test_data)

In [10]:
import pandas as pd
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
import math
from random import randint
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
class Smote():
    def __init__(self,distance,range1,range2):
        self.synthetic_arr = []
        self.newindex = 0
        self.distance_measure = distance
        self.range1 =range1
        self.range2 = range2

    def Populate(self, N, i, indices, min_samples, k):
        """
            此代码主要作用是生成增强数组

            Returns:返回增强后的数组
        """

        choice_list =[]
        def choice(data):
            p = []
            wc = {}
            for num in data:
                # print(num)
                wc[num] = wc.setdefault(num, 0) + 1
            for key in wc.keys():
                p.append(wc[key] / len(data))
            # print(p)
            keylist = np.array([key for key in wc.keys()])
            # print(wc[0])
            return keylist,p
        for index in self.range1:
            choice_list.append(choice(min_samples[:,index]))

        while N != 0:
            arr = np.zeros(len(min_samples[0]))
            arr[-2] = min_samples[i][-2]
            arr[-1] = min_samples[i][-1]
            nn = randint(0, k - 2)
            # 统计离散型变量
            for rowindex,index in enumerate(self.range1):
                arr[index] = np.random.choice(choice_list[rowindex][0],size=1,p=choice_list[rowindex][1])
            #for attr in features2:
            for attr in self.range2:
                min_samples[i][attr] = float(min_samples[i][attr])
                min_samples[indices[nn]][attr] = float(min_samples[indices[nn]][attr])
                try:
                    diff = float(min_samples[indices[nn]][attr]) - float(min_samples[i][attr])
                except:
                    print('这是第%d列'%attr,min_samples[indices[nn]][attr],min_samples[i][attr])
                gap = random.uniform(0, 1)

                arr[attr] = float(min_samples[i][attr]) + gap * diff
            #print(arr)
            self.synthetic_arr.append(arr)
            self.newindex = self.newindex + 1
            N = N - 1

    def k_neighbors(self, euclid_distance, k):
        nearest_idx_npy = np.empty([euclid_distance.shape[0], euclid_distance.shape[0]], dtype=np.int64)

        for i in range(len(euclid_distance)):
            idx = np.argsort(euclid_distance[i])
            nearest_idx_npy[i] = idx
            idx = 0

        return nearest_idx_npy[:, 1:k]

    def find_k(self, X, k):

        """
               Finds k nearest neighbors using euclidian distance

               Returns: The k nearest neighbor
        """

        euclid_distance = np.empty([X.shape[0], X.shape[0]], dtype=np.float32)

        for i in range(len(X)):
            dist_arr = []
            for j in range(len(X)):
                dist_arr.append(math.sqrt(sum((X[j] - X[i]) ** 2)))
            dist_arr = np.asarray(dist_arr, dtype=np.float32)
            euclid_distance[i] = dist_arr

        return self.k_neighbors(euclid_distance, k)

    def generate_synthetic_points(self, min_samples, N, k):

        """

            Parameters
            ----------
            min_samples : 要增强的数据
            N :要额外生成的负样本的数目
            k : int. Number of nearest neighbours.
            Returns
            -------
            S : Synthetic samples. array,
                shape = [(N/100) * n_minority_samples, n_features].
        """

        if N < 1:
            raise ValueError("Value of N cannot be less than 100%")

        if self.distance_measure not in ('euclidian', 'ball_tree'):
            raise ValueError("Invalid Distance Measure.You can use only Euclidian or ball_tree")

        if k > min_samples.shape[0]:
            raise ValueError("Size of k cannot exceed the number of samples.")

        T = min_samples.shape[0]

        if self.distance_measure == 'euclidian':
            indices = self.find_k(min_samples, k)

        elif self.distance_measure == 'ball_tree':
            nb = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(min_samples)
            distance, indices = nb.kneighbors(min_samples)
            indices = indices[:, 1:]

        for i in range(indices.shape[0]):
            self.Populate(N, i, indices[i], min_samples, k)

        return np.asarray(self.synthetic_arr)

In [14]:
rowindex = []
for i, line in enumerate(data):
    if line[-2] ==4.0:
        rowindex.append(i)
range1 = [1,2,3,6,11,13,14,20,21]
range2 = [0,4,5,7,8,9,10,12,15,16,17,18,19,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]
minsamples = data[rowindex]
smote = Smote(distance='ball_tree',range1=range1,range2=range2)
smotedata = smote.generate_synthetic_points(min_samples=minsamples,N=100,k=9)


In [15]:
smotedata.shape

(5200, 43)

In [16]:
adddata = np.concatenate((smotedata,data),axis=0)

In [17]:
adddata.shape

(499221, 43)

In [18]:
fill_nan(test_data)

In [2]:
#  保存数据
"""
np.save('data.npz',data)
np.save('adddata.npz',adddata)
np.save('testdata.npz',test_data)
"""
import numpy as np
import pandas as pd
import codecs
import matplotlib
import matplotlib.pyplot as plt
import random
import random
import codecs
data = np.load('data.npz.npy')
adddata =np.load('adddata.npz.npy')
test_data =np.load('testdata.npz.npy')


In [3]:
def gen_train_test(dat):        #分割训练集和测试集
    index = int(len(dat)*0.8)
    np.random.shuffle(dat)
    traindat=torch.from_numpy(dat[:index,:-2]).float()
    trainlabel=torch.from_numpy(dat[:index,-2]).long()
    validdat = torch.from_numpy(dat[index:,:-2]).float()
    validlabel = torch.from_numpy(dat[index:,-2]).long()
    # print(dat[0])
    return traindat,trainlabel,validdat,validlabel

In [4]:
import tqdm
import torch
from  DBN import DBN
import torch.utils.data as Data
def train_batch(traind,trainl,SIZE=500,SHUFFLE=True):   #分批处理
    trainset=Data.TensorDataset(traind,trainl)
    trainloader=Data.DataLoader(
        dataset=trainset,
        batch_size=SIZE,
        shuffle=SHUFFLE)
    return trainloader


In [5]:
traindat,trainlabel, validdat, validlabel = gen_train_test(data)

In [6]:
trainloader = train_batch(traindat,trainlabel)
from torch.autograd.variable import  Variable

In [7]:
device = torch.device('cuda:0')


In [9]:
import time

start_time = time.time()
dbn=DBN(visible_units=len(traindat[0]))
dbn.to(device)
dbn.train()
dbn.train_static(train_data=traindat,train_labels=trainlabel,num_epochs=3,batch_size=20)

optimizer = torch.optim.SGD(dbn.parameters(), lr=0.001, momentum=0.9)
loss_func = torch.nn.CrossEntropyLoss()

dbn.trainBP(trainloader)

for epoch in range(5):
    for step,(x,y) in tqdm(enumerate(trainloader)):
        # print(x.data.numpy(),y.data.numpy())

        b_x=Variable(x)
        b_y=Variable(y)

        output=dbn(b_x)
        # print(output)
        # print(prediction);print(output);print(b_y)

        loss=loss_func(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step%10==0:
            print('Epoch: ', epoch, 'step:',step,'| train loss: %.4f' % loss.data.numpy())
duration=time.time()-start_time

dbn.eval()
test_x = Variable(validdat);test_y = Variable(validlabel)
test_out = dbn(test_x)
# print(test_out)
test_pred = torch.max(test_out, 1)[1]
pre_val = test_pred.data.squeeze().numpy()
y_val = test_y.data.squeeze().numpy()
print('prediciton:',pre_val);print('true value:',y_val)
accuracy = float((pre_val == y_val).astype(int).sum()) / float(test_y.size(0))
print('test accuracy: %.2f' % accuracy,'duration:%.4f' % duration)


--------------------

Training the 1 st rbm layer

C:\Users\Administrator\AppData\Local\Programs\Python\Python36\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch Error(epoch:0) : 45406564.0000

<class 'torch.Tensor'>

<class 'torch.Tensor'>

<class 'torch.Tensor'>

<class 'torch.Tensor'>

<class 'torch.utils.data.dataloader.DataLoader'>

--------------------

Training the 1 st rbm layer

C:\Users\Administrator\AppData\Local\Programs\Python\Python36\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


KeyboardInterrupt: 